In [ ]:
import os
import json
from kaggle.api.kaggle_api_extended import KaggleApi

kaggle_json_path = "../kaggle.json"

with open(kaggle_json_path, 'r') as file:
    kaggle_creds = json.load(file)

os.environ["KAGGLE_USERNAME"] = kaggle_creds["username"]
os.environ["KAGGLE_KEY"] = kaggle_creds["key"]

dataset_name = 'arshkon/linkedin-job-postings'
download_path = "../data/raw"
os.makedirs(download_path, exist_ok=True)

api = KaggleApi()
api.authenticate()
print("Téléchargement en cours...")
api.dataset_download_files(dataset_name, path=download_path, unzip=True)
print(f"Dataset téléchargé et extrait dans : {download_path}")